## Preproccesing data

In [1]:
# Importing libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle

from sklearn.model_selection import train_test_split, cross_val_score, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import make_scorer, accuracy_score 
from sklearn.svm import SVC 
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 

In [2]:
#function for merging left and right dataframe
def merge(left, right, name ):
    pvs= pd.concat([left, right], axis=1)
    length= len(left)

    # Renaming labels
    pvs.columns = [
    'timestamp', 'acc_x_dashboard_l', 'acc_y_dashboard_l', 'acc_z_dashboard_l',
    'acc_x_above_suspension_l', 'acc_y_above_suspension_l', 'acc_z_above_suspension_l', 
    'acc_x_below_suspension_l', 'acc_y_below_suspension_l', 'acc_z_below_suspension_l', 
    'gyro_x_dashboard_l', 'gyro_y_dashboard_l', 'gyro_z_dashboard_l', 'gyro_x_above_suspension_l', 
    'gyro_y_above_suspension_l', 'gyro_z_above_suspension_l', 'gyro_x_below_suspension_l', 
    'gyro_y_below_suspension_l', 'gyro_z_below_suspension_l', 'mag_x_dashboard_l', 'mag_y_dashboard_l', 
    'mag_z_dashboard_l', 'mag_x_above_suspension_l', 'mag_y_above_suspension_l', 'mag_z_above_suspension_l', 
    'temp_dashboard_l', 'temp_above_suspension_l', 'temp_below_suspension_l', 'timestamp_gps', 
    'latitude', 'longitude', 'speed', 'timestamp', 'acc_x_dashboard_r', 'acc_y_dashboard_r', 
    'acc_z_dashboard_r', 'acc_x_above_suspension_r', 'acc_y_above_suspension_r', 
    'acc_z_above_suspension_r', 'acc_x_below_suspension_r', 'acc_y_below_suspension_r', 
    'acc_z_below_suspension_r', 'gyro_x_dashboard_r', 'gyro_y_dashboard_r', 'gyro_z_dashboard_r', 
    'gyro_x_above_suspension_r', 'gyro_y_above_suspension_r', 'gyro_z_above_suspension_r', 
    'gyro_x_below_suspension_r', 'gyro_y_below_suspension_r', 'gyro_z_below_suspension_r', 
    'mag_x_dashboard_r', 'mag_y_dashboard_r', 'mag_z_dashboard_r', 'mag_x_above_suspension_r', 
    'mag_y_above_suspension_r', 'mag_z_above_suspension_r', 'temp_dashboard_r', 'temp_above_suspension_r', 
    'temp_below_suspension_r', 'timestamp_gps', 'latitude', 'longitude', 'speed'
    ]
    

    # Remove duplicate columns (those with the same name, e.g., 'timestamp_gps', 'latitude', etc.)
    pvs_removed = pvs.loc[:, ~pvs.columns.duplicated()]

    # Columns to keep. We try these first
    to_keep=["timestamp",
             "acc_x_dashboard_l",
             "acc_y_dashboard_l",
             "acc_z_dashboard_l",
             "speed",
             "gyro_x_dashboard_l",
             "gyro_y_dashboard_l",
             "gyro_z_dashboard_l"
            ]
    
    pvs_removed=pvs_removed[to_keep]

    #create the driver column ( target )
    if name in ("pvs1_gps_mpu","pvs2_gps_mpu","pvs3_gps_mpu"):
        pvs_removed['Driver'] = 1
    elif name in ("pvs4_gps_mpu","pvs5_gps_mpu","pvs6_gps_mpu"):
        pvs_removed['Driver'] = 2
    else: 
        pvs_removed['Driver'] = 3
        
    return pvs_removed

#Drivers do not drive in the start and end of each route. Function for removing these indicies
def remove_zero_values(dfs):

    new = dfs

    non_zero_indices = {}
    threshold = 0.1  # Define the threshold to detect non-zero speeds

    for key, df in new.items():

        first_non_zero_index = (df['speed'].abs() > threshold).idxmax()
        last_non_zero_index = (df['speed'][::-1].abs() > threshold).idxmax()

        non_zero_indices[key] = (first_non_zero_index, last_non_zero_index)

    #print(non_zero_indices)

    # Iterate through each dataset and its respective non-zero index
    for key, index in non_zero_indices.items():
        new[key] = new[key].iloc[index[0]:index[1]].reset_index(drop=True)  # Remove rows up to the index and reset index

    return new

#Function for making dfs equal size in order to avoid bias. Each route is truncated to the same length
def equalize_dfs(dfs):

    new = dfs

    # Split DataFrames into three routes
    dfs_keys = list(new.keys())  # Get all keys in the dictionary
    first_route_keys = dfs_keys[:3]  # First three keys for the first route
    second_route_keys = dfs_keys[3:6]  # Next three keys for the second route
    third_route_keys = dfs_keys[6:]  # Last three keys for the third route

    # Calculate the minimum lengths for each route
    min_length_first_route = min(len(new[key]) for key in first_route_keys)
    min_length_second_route = min(len(new[key]) for key in second_route_keys)
    min_length_third_route = min(len(new[key]) for key in third_route_keys)

    #print(f"Shortest length in the first route: {min_length_first_route}")
    #print(f"Shortest length in the second route: {min_length_second_route}")
    #print(f"Shortest length in the third route: {min_length_third_route}")

    # Truncate all DataFrames in each route to the respective minimum length
    for key in first_route_keys:
        new[key] = new[key].iloc[:min_length_first_route].reset_index(drop=True)

    for key in second_route_keys:
        new[key] = new[key].iloc[:min_length_second_route].reset_index(drop=True)

    for key in third_route_keys:
        new[key] = new[key].iloc[:min_length_third_route].reset_index(drop=True)

    return new

#Need to make the ratio between dfs and ratio to be an integer in order for dataframe reduction to work
def round_dfs(dfs, ratio):

    new = dfs

    for key, df in new.items():

        rounded_length = len(df) - (len(df) % ratio)

        new[key] = df.iloc[:rounded_length].reset_index(drop=True)

    return new


def reduce_df(df, ratio, name):
    new = pd.DataFrame()
    print(f"Reducing {name}")
    
    # Iterate in steps of 'ratio' through the DataFrame
    for i in range(0, len(df) - ratio + 1, ratio):
        # Select the rows from i to i+ratio (inclusive of i, exclusive of i+ratio)
        subset = df.iloc[i:i+ratio]
        
        # Calculate the mean for each column in the selected rows
        merged_row = subset.mean(axis=0)
        
        # Append the result as a new row to the DataFrame `new`
        new = pd.concat([new, merged_row.to_frame().T], ignore_index=True)
        
    return new


def delete_timestamp(df):
    to_remove=["timestamp_max","timestamp_min","timestamp_mean","timestamp_STD"]
    new= df.drop(columns=to_remove, axis=1, errors='ignore')

    return new

def update_name(df):
    new_column_names = [ 'timestamp_max','acc_x_dashboard_l_max', 'acc_y_dashboard_l_max', 
        'acc_z_dashboard_l_max', 'speed_max', 'gyro_x_dashboard_l_max', 
        'gyro_y_dashboard_l_max', 'gyro_z_dashboard_l_max', 
        'timestamp_min', 'acc_x_dashboard_l_min', 'acc_y_dashboard_l_min', 
        'acc_z_dashboard_l_min', 'speed_min', 'gyro_x_dashboard_l_min', 
        'gyro_y_dashboard_l_min', 'gyro_z_dashboard_l_min', 
        'timestamp_mean', 'acc_x_dashboard_l_mean', 'acc_y_dashboard_l_mean', 
        'acc_z_dashboard_l_mean', 'speed_mean', 'gyro_x_dashboard_l_mean', 
        'gyro_y_dashboard_l_mean', 'gyro_z_dashboard_l_mean', 
        'timestamp_STD_', 'acc_x_dashboard_l_STD_', 'acc_y_dashboard_l_STD_', 
        'acc_z_dashboard_l_STD_', 'speed_STD_', 'gyro_x_dashboard_l_STD_', 
        'gyro_y_dashboard_l_STD_', 'gyro_z_dashboard_l_STD_',
         'acc_x_dashboard_l_jerk', 'acc_y_dashboard_l_jerk', 
        'acc_z_dashboard_l_jerk', 'speed_jerk', 'gyro_x_dashboard_l_jerk', 
        'gyro_y_dashboard_l_jerk', 'gyro_z_dashboard_l_jerk'
    ]
    df.columns = new_column_names
    return df


#function for computing the rolling window and the aggregation functions, 
#--> returns a disctionary containing the 9 dataframes winodwed (still separated)
def computeWindow(name,df, windowSize=100, show=False):
    if show:
        print("--------------------------")
        print(f"computing dataset {name}")
    
    #define empty dataframe  

    X= df.iloc[:, :-1]
    Y= df.iloc[:len(df)-windowSize+1, -1]
    windowed_df=pd.DataFrame()

    #define starting and ending index
    for start_idx in range(len(df)-windowSize+1):
        end_idx=start_idx+windowSize

        #extract rows belonging to the window
        window=X.iloc[start_idx:end_idx]

        #take beginning timestamp and ending timestamp
        start_timestamp=window.iloc[0,0]
        end_timestamp=window.iloc[-1,0]

        
        #keep only sensor data
        sensor_data_window=window.iloc[:,1:]

        #compute metrics for the specific window
        max_values=window.max()
        min_values=window.min()
        mean_values=window.mean()
        std_values=window.std()
        jerk_values=(sensor_data_window.iloc[-1]-sensor_data_window.iloc[0])/(end_timestamp-start_timestamp)

        #concate them (place side by side)
        new_row=pd.concat([max_values,min_values,mean_values,std_values,jerk_values])
        new_row = new_row.to_frame().T 

        windowed_df = pd.concat([windowed_df, new_row], ignore_index=True)

        
        if show and start_idx%10000==0:
            print(start_idx)

    #place aside again  X and Y
    final=pd.concat([delete_timestamp(update_name(windowed_df)),Y], axis=1)  
    return final

In [3]:
# Importing datasets
filepaths_left={
    "pvs1_gps_mpu" : r"../archive/PVS 1/dataset_gps_mpu_left.csv",
    "pvs2_gps_mpu" : r"../archive/PVS 2/dataset_gps_mpu_left.csv",
    "pvs3_gps_mpu" : r"../archive/PVS 3/dataset_gps_mpu_left.csv",
    "pvs4_gps_mpu" : r"../archive/PVS 4/dataset_gps_mpu_left.csv",
    "pvs5_gps_mpu" : r"../archive/PVS 5/dataset_gps_mpu_left.csv",
    "pvs6_gps_mpu" : r"../archive/PVS 6/dataset_gps_mpu_left.csv",
    "pvs7_gps_mpu" : r"../archive/PVS 7/dataset_gps_mpu_left.csv",
    "pvs8_gps_mpu" : r"../archive/PVS 8/dataset_gps_mpu_left.csv",
    "pvs9_gps_mpu" : r"../archive/PVS 9/dataset_gps_mpu_left.csv"
}

filepaths_right={
    "pvs1_gps_mpu" : r"../archive/PVS 1/dataset_gps_mpu_right.csv",
    "pvs2_gps_mpu" : r"../archive/PVS 2/dataset_gps_mpu_right.csv",
    "pvs3_gps_mpu" : r"../archive/PVS 3/dataset_gps_mpu_right.csv",
    "pvs4_gps_mpu" : r"../archive/PVS 4/dataset_gps_mpu_right.csv",
    "pvs5_gps_mpu" : r"../archive/PVS 5/dataset_gps_mpu_right.csv",
    "pvs6_gps_mpu" : r"../archive/PVS 6/dataset_gps_mpu_right.csv",
    "pvs7_gps_mpu" : r"../archive/PVS 7/dataset_gps_mpu_right.csv",
    "pvs8_gps_mpu" : r"../archive/PVS 8/dataset_gps_mpu_right.csv",
    "pvs9_gps_mpu" : r"../archive/PVS 9/dataset_gps_mpu_right.csv"


}

# Left and right corresponds to sensordata from either side of the cars
datasets_left = {name:pd.read_csv(path) for name,path in filepaths_left.items()}
datasets_right = {name:pd.read_csv(path) for name,path in filepaths_right.items()}

datasets = {name: merge(datasets_left[name],datasets_right[name], name) for name in datasets_right.keys()}

In [5]:
datasets_rm_zeros = remove_zero_values(datasets)
equalized_datasets = equalize_dfs(datasets_rm_zeros)  

In [12]:
#REDUCE THE DATA
ratio=40
rounded_datasets = round_dfs(equalized_datasets, ratio)

#for key, index in rounded_datasets.items():
#    print("Length of speed list for", key, ": ", len(rounded_datasets[key]['speed']))

datasets_reduced={name:reduce_df(rounded_datasets[name],ratio,name) for name in rounded_datasets.keys()}

Reducing pvs1_gps_mpu
Reducing pvs2_gps_mpu
Reducing pvs3_gps_mpu
Reducing pvs4_gps_mpu
Reducing pvs5_gps_mpu
Reducing pvs6_gps_mpu
Reducing pvs7_gps_mpu
Reducing pvs8_gps_mpu
Reducing pvs9_gps_mpu


In [13]:
# Save the dictionary with DataFrames
with open("datasets_reduced.pkl", "wb") as file:
    pickle.dump(datasets_reduced, file)

In [14]:
#compute all windowed dfs in order to store them : 
window_sizes=[100,200]
for size in window_sizes:
    dfs_windowed={name:computeWindow(name,datasets_reduced[name],size, True) for name in datasets_reduced.keys()}
    # Save the dictionary with DataFrames
    with open(f"dfs_windowed_{size}.pkl", "wb") as file:
        pickle.dump(dfs_windowed, file)
    

--------------------------
computing dataset pvs1_gps_mpu
0
--------------------------
computing dataset pvs2_gps_mpu
0
--------------------------
computing dataset pvs3_gps_mpu
0
--------------------------
computing dataset pvs4_gps_mpu
0
--------------------------
computing dataset pvs5_gps_mpu
0
--------------------------
computing dataset pvs6_gps_mpu
0
--------------------------
computing dataset pvs7_gps_mpu
0
--------------------------
computing dataset pvs8_gps_mpu
0
--------------------------
computing dataset pvs9_gps_mpu
0
--------------------------
computing dataset pvs1_gps_mpu
0
--------------------------
computing dataset pvs2_gps_mpu
0
--------------------------
computing dataset pvs3_gps_mpu
0
--------------------------
computing dataset pvs4_gps_mpu
0
--------------------------
computing dataset pvs5_gps_mpu
0
--------------------------
computing dataset pvs6_gps_mpu
0
--------------------------
computing dataset pvs7_gps_mpu
0
--------------------------
computing dat

In [9]:
#create the full df
test_track=3
threshold_pca=0.6
window_sizes=[100,200,300]
model_type="lr"
num_folds=5

tuning_and_evaluation(datasets_reduced,window_sizes,model_type,num_folds,test_track,threshold_pca)

NameError: name 'tuning_and_evaluation' is not defined